In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download('punkt')

from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /home/emmanuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
class Node:
    def __init__(self, item):
        self.item = item
        self.next = None

class LL:
    def __init__(self):
        self.head = None
    # method to insert node with sorting
    def add_node(self,val):
        newNode = Node(val)
        if not self.head:
            # If the list is empty, set the new node as the head
            self.head = newNode
            return
        temp = self.head
        while temp:
            if val == temp.item:
                return
            temp = temp.next
        if self.head.item > val:
            # If the new value is smaller than the head, insert it at the beginning
            newNode.next = self.head
            self.head = newNode
            return
        temp = self.head
        while temp and temp.next:
            if val < temp.next.item:
                # Find the appropriate position to insert the new node
                break
            temp = temp.next
        newNode.next = temp.next
        temp.next = newNode
                    
    def print_list(self):
        # Traverse the linked list and print its items
        tmp = self.head
        while tmp:
            print(tmp.item, end=' ')
            tmp = tmp.next

In [4]:
import pickle

#Load the inverted and positional index and a file that contains all document names

with open("inverted_index.pkl", "rb") as f:
    inverted_index= pickle.load(f)

with open("positional_index.pkl", "rb") as f:
    positional_index = pickle.load(f)

with open("docs.pkl", "rb") as f:
    docs = pickle.load(f)

In [5]:
import re
operators = set(['AND', 'OR'])
def shuntingYard(tokens):
    # Implements the Shunting-yard algorithm to convert an infix expression to postfix.
 
    operatorSTK = []

    precedence = {'(': 0, 'AND': 2, 'OR': 1, ')': 3}
    outputQueue = []

    for token in tokens:
        if token == 'NOT':
            operatorSTK.append(token)
        elif token in operators:
            while operatorSTK and precedence[token] <= precedence[operatorSTK[-1]]:
                outputQueue.append(operatorSTK.pop())
            operatorSTK.append(token)
        elif token == "(":
            operatorSTK.append(token)
        elif token == ")":
            while operatorSTK[-1] != "(":
                outputQueue.append(operatorSTK.pop())
            operatorSTK.pop()
        else:
            outputQueue.append(token)

    while operatorSTK:
        outputQueue.append(operatorSTK.pop())

    return outputQueue

def applyOR(operand1, operand2):
    # Applies union on two sets of documents.
    setResult = set()
    if type(operand1) == str:
        ref = inverted_index.get(ps.stem(operand1))
        if ref:
            tmp = ref.head
            while tmp:
                setResult.add(tmp.item)
                tmp = tmp.next
    else:
        setResult = setResult.union(operand1)

    if type(operand2) == str:
        ref = inverted_index.get(ps.stem(operand2))
        if ref:
            tmp = ref.head
            while tmp:
                setResult.add(tmp.item)
                tmp = tmp.next
    else:
        setResult = setResult.union(operand2)
    return setResult

def applyAND(operand1, operand2):
    # Applies intersection on two sets of documents.
    setResult = set()
    if type(operand1) == str:
        ref = inverted_index.get(ps.stem(operand1))
        if ref:
            set1 = set()
            tmp = ref.head
            while tmp:
                set1.add(tmp.item)
                tmp = tmp.next
            setResult = setResult.union(set1)
    else:
        setResult = setResult.union(operand1)

    if type(operand2) == str:
        ref = inverted_index.get(ps.stem(operand2))
        if ref:
            set2 = set()
            tmp = ref.head
            while tmp:
                set2.add(tmp.item)
                tmp = tmp.next
            setResult = setResult.intersection(set2)
    else:
        setResult = setResult.intersection(operand2)
    return setResult

def applyNOT(operand):
    setResult = set()
    if type(operand) == str:
        ref = inverted_index.get(ps.stem(operand))
        if ref:
            set1 = set()
            tmp = ref.head
            while tmp:
                set1.add(tmp.item)
                tmp = tmp.next
            setResult = docs.difference(set1)
    else:
        setResult = docs.difference(operand)
    return setResult
            

def evaluate(postfix):
    operandSTK = []
    operators = set(['AND', 'OR', 'NOT'])

    if len(postfix) == 1:
        result = inverted_index.get(postfix[0])
        setResult = set()
        if result:
            tmp = result.head
            while tmp:
                setResult.add(tmp.item)
                tmp = tmp.next
        return list(setResult)
    
    for token in postfix:
        if token == 'OR':
            operand1 = operandSTK.pop()
            operand2 = operandSTK.pop()
            result = applyOR(operand1, operand2)
            operandSTK.append(result)
        elif token == 'AND':
            operand1 = operandSTK.pop()
            operand2 = operandSTK.pop()
            result = applyAND(operand1, operand2)
            operandSTK.append(result)
        elif token == 'NOT':
            if operandSTK and operandSTK[-1] == 'NOT':
                operandSTK.pop()
                operand = operandSTK.pop()
                result = operand
            else:
                operand = operandSTK.pop()
                result = applyNOT(operand)
            operandSTK.append(result)
        else:
            operandSTK.append(token)
    
    return list(operandSTK[0])
def process_boolean_query(query):
    oper = set(['AND', 'OR', 'NOT'])

    # Split the query into tokens
    tokens = re.findall(r"[\w]+|[^-_\w\s()@#$%^&*+={[\]};,<>./?~`\"]", query)
    # Lower case tokens
    tokens = [token.lower() if token not in oper else token for token in tokens]
    # Get postfix using shunting yard algorithm
    postfix = shuntingYard(tokens)
    result = evaluate(postfix)
    return result

In [6]:
import re
def evalPosQuery(term1, term2, k):
    # Finds documents where two terms appear within a specified distance (k) of each other.
    answer = []
    def posIntersect(p1, p2, k, doc):
        l = []
        pp1 = p1.head
        pp2 = p2.head
        while pp1 is not None:
            while pp2 is not None:
                if abs(pp1.item - pp2.item) <= k:
                    l.append(pp2.item)
                elif pp2.item > pp1.item:
                    break
                pp2 = pp2.next
            while l != [] and abs(l[0] - pp1.item) > k:
                l.pop(0)
            for ps in l:
                answer.append([doc, pp1.item, ps])
            pp1 = pp1.next
    # Check if terms exist in the positional index and iterate through documents
    if positional_index.get(term1) and positional_index.get(term2):
        for doc in docs:
            if positional_index[term1].get(doc) and positional_index[term2].get(doc):
                posIntersect(positional_index[term1][doc], positional_index[term2][doc], k, doc)
            else:
                continue # Skip documents which do not contain both terms
    return answer

In [ ]:
import tkinter as tk
from tkinter import ttk
import re

# Function to process proximity queries
def process_proximity_query(query):
    # Split the query into tokens
    tkns = query.split(' ')
    
    if len(tkns) == 3:
        answer = evalPosQuery(ps.stem(tkns[0]), ps.stem(tkns[1]), int(re.search(r'\d+', tkns[2]).group()))
        return answer

# Function to display results in the specified treeview
def display_results(result, treeview):
    # Clear existing results in the treeview
    for item in treeview.get_children():
        treeview.delete(item)

    if result:
        # Display the new results
        for i, (doc, first_idx, second_idx) in enumerate(result):
            data = (doc, first_idx, second_idx)
            treeview.insert(parent='', index=i, values=data)
    # Add to show end of table
    treeview.insert(parent='', index='end', values=('XXX', 'YYY', 'ZZZ'))

def boolean_results(result, treeview):
    for item in treeview.get_children():
        treeview.delete(item)
    if result:
        for i, doc in enumerate(result):
            data = (doc)
            treeview.insert(parent='', index=i, values=data)
    # Add to show end of table
    treeview.insert(parent='', index='end', values=('XXX'))

# Function to handle search button click for proximity queries
def search_proximity():
    query = entry_proximity.get()
    result = process_proximity_query(query)
    display_results(result, treeview_proximity)

# Function to handle search button click for boolean queries
def search_boolean():
    query = entry_boolean.get()
    result = process_boolean_query(query)
    boolean_results(result, treeview_boolean)

# Create the main Tkinter window
root = tk.Tk()
root.title("Proximity and Boolean Query Search")

# Section for proximity queries
frame_proximity = ttk.Frame(root)
frame_proximity.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

label_proximity = ttk.Label(frame_proximity, text="Proximity Query:")
label_proximity.grid(row=0, column=0, padx=5, pady=5)

entry_proximity = ttk.Entry(frame_proximity, width=30)
entry_proximity.grid(row=0, column=1, padx=5, pady=5)

button_proximity = ttk.Button(frame_proximity, text="Search", command=search_proximity)
button_proximity.grid(row=0, column=2, padx=5, pady=5)

treeview_proximity = ttk.Treeview(frame_proximity, columns=('doc', 'first', 'second'), show='headings')
treeview_proximity.heading('doc', text='Document ID')
treeview_proximity.heading('first', text='First word index')
treeview_proximity.heading('second', text='Second word index')
treeview_proximity.grid(row=1, column=0, columnspan=3, sticky="nsew")

# Section for boolean queries
frame_boolean = ttk.Frame(root)
frame_boolean.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

label_boolean = ttk.Label(frame_boolean, text="Boolean Query:")
label_boolean.grid(row=0, column=0, padx=5, pady=5)

entry_boolean = ttk.Entry(frame_boolean, width=80)
entry_boolean.grid(row=0, column=1, padx=5, pady=5)

button_boolean = ttk.Button(frame_boolean, text="Search", command=search_boolean)
button_boolean.grid(row=0, column=2, padx=5, pady=5)

treeview_boolean = ttk.Treeview(frame_boolean, columns=('doc'), show='headings')
treeview_boolean.heading('doc', text='Document ID')
treeview_boolean.grid(row=1, column=0, columnspan=3, sticky="nsew")

# Set row and column weights for resizing
root.grid_rowconfigure(0, weight=1)
root.grid_rowconfigure(1, weight=1)
root.grid_columnconfigure(0, weight=1)

# Start the Tkinter event loop
root.mainloop()